Ось **детальний опис REST API** та пояснення, **як реалізувати REST API за допомогою FastAPI** — з прикладами та поясненнями українською.

---

## 🔷 Що таке REST API?

### 💡 REST (Representational State Transfer) — це стиль архітектури веб-сервісів, заснований на HTTP-протоколі.

REST API — це **інтерфейс**, через який клієнти (наприклад, frontend, мобільні додатки або інші сервіси) можуть обмінюватися даними з сервером.

---

## 📑 Основні принципи REST

| Принцип               | Пояснення                                                                 |
|-----------------------|---------------------------------------------------------------------------|
| 🔄 Stateless           | Кожен запит не має "пам’яті" — вся потрібна інформація передається в запиті |
| 🌐 URI                 | Кожен ресурс має унікальну адресу (наприклад `/users/1`)                 |
| 📦 Використання HTTP  | HTTP-методи: GET, POST, PUT, DELETE, PATCH                                |
| 🧱 Формат              | Зазвичай JSON (може бути XML тощо)                                        |

---

## 🌐 REST API: типові операції (CRUD)

| Операція | HTTP-метод | URL             | Дія                          |
|----------|------------|------------------|-------------------------------|
| Create   | POST       | `/items/`        | Створити новий об'єкт        |
| Read     | GET        | `/items/`        | Отримати список               |
| Read     | GET        | `/items/{id}`    | Отримати конкретний об'єкт   |
| Update   | PUT        | `/items/{id}`    | Повністю оновити             |
| Patch    | PATCH      | `/items/{id}`    | Частково оновити             |
| Delete   | DELETE     | `/items/{id}`    | Видалити                     |

---

## 🚀 Як реалізувати REST API у FastAPI?

### 📁 Структура:

```
rest_api_app/
├── main.py
├── models.py
├── schemas.py
├── database.py
└── requirements.txt
```

---

### 📄 `requirements.txt`

```
fastapi
uvicorn
sqlalchemy
pydantic
```

---

### 📄 `database.py`

```python
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

DATABASE_URL = "sqlite:///./items.db"

engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False)
Base = declarative_base()
```

---

### 📄 `models.py`

```python
from sqlalchemy import Column, Integer, String
from database import Base

class Item(Base):
    __tablename__ = "items"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    description = Column(String)
```

---

### 📄 `schemas.py`

```python
from pydantic import BaseModel

class ItemCreate(BaseModel):
    name: str
    description: str

class ItemOut(ItemCreate):
    id: int

    class Config:
        orm_mode = True
```

---

### 📄 `main.py`

```python
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session
import models, schemas
from database import engine, SessionLocal

models.Base.metadata.create_all(bind=engine)

app = FastAPI()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Create
@app.post("/items/", response_model=schemas.ItemOut)
def create_item(item: schemas.ItemCreate, db: Session = Depends(get_db)):
    db_item = models.Item(**item.dict())
    db.add(db_item)
    db.commit()
    db.refresh(db_item)
    return db_item

# Read All
@app.get("/items/", response_model=list[schemas.ItemOut])
def read_items(db: Session = Depends(get_db)):
    return db.query(models.Item).all()

# Read by ID
@app.get("/items/{item_id}", response_model=schemas.ItemOut)
def read_item(item_id: int, db: Session = Depends(get_db)):
    item = db.query(models.Item).get(item_id)
    if not item:
        raise HTTPException(status_code=404, detail="Item not found")
    return item

# Update
@app.put("/items/{item_id}", response_model=schemas.ItemOut)
def update_item(item_id: int, updated: schemas.ItemCreate, db: Session = Depends(get_db)):
    item = db.query(models.Item).get(item_id)
    if not item:
        raise HTTPException(status_code=404, detail="Item not found")
    item.name = updated.name
    item.description = updated.description
    db.commit()
    db.refresh(item)
    return item

# Delete
@app.delete("/items/{item_id}")
def delete_item(item_id: int, db: Session = Depends(get_db)):
    item = db.query(models.Item).get(item_id)
    if not item:
        raise HTTPException(status_code=404, detail="Item not found")
    db.delete(item)
    db.commit()
    return {"ok": True, "deleted_id": item_id}
```

---

## ✅ Як це працює?

1. **Запускаєш API**:
```bash
uvicorn main:app --reload
```

2. **Відкриваєш документацію**:
[http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs) — інтерфейс Swagger

3. **Працюєш з API** — можеш:
   - створювати нові `items`
   - отримувати список
   - оновлювати
   - видаляти

---

## 🧠 Переваги FastAPI для REST API

- 🚀 **Швидкість розробки**
- 🧪 **Автотестування через Swagger**
- ✅ **Вбудована валідація**
- 📘 **Автогенерація документації**
- 🔧 **Підтримка асинхронності**

---

Хочеш розширити цей приклад (авторизація, пагінація, фільтрація, async-база)? Напиши — зробимо крок за кроком.